In [1]:
from vowpalwabbit import pyvw

In [7]:
# Property 1: Inital test to verify that more than one vw can be used
# and that they do not share model memory

first_vw = pyvw.vw(quiet=True)
second_vw = pyvw.vw(quiet=True)
print(f"first vw has {hex(id(first_vw))} and second vw has {hex(id(second_vw))}")
id(first_vw) == id(second_vw), "First and second instances point to the same memory!"

# But they don't seem to share model memory? Note how even the examples are in scope of
# the instance
ex = first_vw.example('1 | a b c')
first_vw.learn(ex)

first_predict = first_vw.predict(ex)
second_predict = second_vw.predict(ex)

print(f"first model predicted {first_predict}, second predicted  {second_predict}")
assert first_predict != second_predict, "Models seems equivalent because same prediction was given"

first vw has 0x7f11cc42d950 and second has 0x7f11cc3b1ae0
first model predicted 0.632030725479126, second predicted  0.0


In [ ]:
# Property 2: Referencing a config file, create a pool of learners